In [1]:
import os, itables, h3pandas, pygris, folium, numba, pathlib
import pandas as pd
import geopandas as gpd
import contextily as ctx
from matplotlib import pyplot as plt
itables.options.style = 'float:left;width:auto'

from pprint import pprint

In [2]:
df = pd.read_parquet(os.path.join('data', 'parsedData.parquet')).reset_index(drop=True)

In [3]:
df['bead'] = (df.cableServed.astype('str') + ',' +
    df.copperServed.astype('str') + ',' +
    df.fiberServed.astype('str') + ',' +
    df.lfwServed.astype('str')
)

In [4]:
itables.show(df)

In [5]:
waCountyFipsList = df.county_geoid.unique().tolist()

In [6]:
waCounties = gpd.GeoDataFrame(pygris.counties(state='WA', cache=True))

Using the default year of 2021
Using FIPS code '53' for input 'WA'


In [7]:
itables.show(waCounties)

In [8]:
for countyFips in waCountyFipsList:
    # print(countyFips)
    county = waCounties[waCounties.GEOID == countyFips].to_dict('records')[0]
    countyDf = df[df.county_geoid == countyFips]
    pprint(county['NAME'])
    # itables.show(countyDf)
    lfwServedCounty = countyDf[countyDf.bead == 'Unserved,Unserved,Unserved,Served'].h3_res8_id.value_counts().reset_index().rename(columns={'count': 'lfwBsls'})
    countyStats = countyDf.h3_res8_id.value_counts().reset_index().rename(columns={'count': 'bsls'})
    # itables.show(lfwServedCounty)
    # itables.show(countyStats)
    lfwServedCounty = lfwServedCounty.merge(countyStats, how='left')
    lfwServedCounty['lfwPercent'] = (lfwServedCounty.lfwBsls / lfwServedCounty.bsls) * 100
    # itables.show(lfwServedCounty)
    lfwServedCounty = lfwServedCounty.set_index('h3_res8_id')
    lfwServedCounty = lfwServedCounty.h3.h3_to_geo_boundary()
    lfwServedCounty = lfwServedCounty.reset_index()
    lfwServedCounty = gpd.GeoDataFrame(lfwServedCounty)
    lfwServedCounty = lfwServedCounty.to_crs(epsg=3857)
    # itables.show(lfwServedCounty)
    # lfwServedCounty.plot()
    countyBorder = waCounties[waCounties.GEOID == countyFips].to_crs(epsg=3857)
 
    if len(lfwServedCounty) != 0:
        fig, ax2 = plt.subplots(figsize=(18,15))
        if len(lfwServedCounty) < 5 :
            classificationScheme = None
            percentScheme = None
        else:
            classificationScheme = 'FisherJenks'
            percentScheme = 'equalinterval'
        countyBorder.plot(ax=ax2, facecolor='none', edgecolor = 'black', lw=3)
        lfwServedCounty.plot(ax=ax2, column='lfwBsls', scheme=classificationScheme, legend=True)
        ctx.add_basemap(ax=ax2, crs='epsg:3857', source=ctx.providers.CartoDB.Positron, zoom=10)
        ax2.axis('off')
        fig.savefig(pathlib.Path('exports', 'pngs', f'{county["NAME"]}.png'), bbox_inches='tight')
        plt.close()
        m = folium.Map(tiles=None)
        folium.TileLayer('CartoDB Positron', name='Positron').add_to(m)
        lfwServedCounty.explore(m = m, column='lfwBsls', scheme=classificationScheme, name='BSLs with only LFW', show=False)
        lfwServedCounty.explore(m = m, column='lfwPercent', scheme=percentScheme, name='Percent of LFW only BSLs')
        countyBorder.explore(m = m, style_kwds={'fill': False, 'color': 'black'}, name=f'{county["NAME"]} County Border')
        m.fit_bounds(m.get_bounds())
        folium.LayerControl().add_to(m)
        m.save(os.path.join('exports', 'html', f'{county["NAME"]}.html'))
    else:
        pass

'Stevens'
'Kitsap'
'Pacific'
'Cowlitz'
'Lewis'
'Island'
'Thurston'
'Mason'
'Whatcom'
'Skagit'
'Pierce'
'Grays Harbor'
'Snohomish'
'Spokane'
'Clark'
'King'
'Yakima'
'Clallam'
'Asotin'
'Jefferson'
'Chelan'
'Skamania'
'Kittitas'
'Okanogan'
'Walla Walla'
'Douglas'
'Whitman'
'Franklin'
'Grant'
'Benton'
'Adams'
'San Juan'
'Pend Oreille'
'Ferry'
'Klickitat'
'Wahkiakum'
'Columbia'
'Lincoln'
'Garfield'
